In [1]:
import sys
rootDir = "../../"
sys.path.append(rootDir + "Analysis/Util/")
sys.path.append(rootDir + "Database/")

from DatabaseAPI import DatabaseAPI
import numpy as np
from get_time_interval_data import *

from Mapping import *
from PCA import *
import pickle

for stuff in sys.path:
    print(stuff)

/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload

/usr/local/lib/python3.6/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.6/dist-packages/IPython/extensions
/root/.ipython
../../Analysis/Util/
../../Database/
../../Database/Tables
../
../
../../Analysis/Util


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F

'''
simpleMapper() maps the following:
DOA (Direction of Arrival) -> Real Room Space (x,y,z,etc.)
according to the following parameters:
- numDOA : Number of DOAs (depending on number of arrays)
- outDim : Number of Dimensions (2 for 2D space, 3 for 3D space, etc.)
'''

class classificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

class simpleMapper():
    def __init__(self, numDOA=15, outDim=2):
        super().__init__()
        H = 64
        self.network = torch.nn.Sequential(
                torch.nn.Linear(numDOA, H),
                torch.nn.ReLU(),
                torch.nn.Linear(H, outDim),
            )
#         self.network = nn.Sequential(
            
#             nn.Conv1d(numDOA, 32, kernel_size = 3, padding = 1),
#             nn.ReLU(),
#             nn.Conv1d(32,64, kernel_size = 3, stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.MaxPool1d(2,2),
        
#             nn.Conv1d(64, 128, kernel_size = 3, stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.Conv1d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.MaxPool1d(2,2),
            
#             nn.Conv1d(128, 256, kernel_size = 3, stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.Conv1d(256,256, kernel_size = 3, stride = 1, padding = 1),
#             nn.ReLU(),
#             nn.MaxPool1d(2,2),
            
#             nn.Flatten(),
#             nn.Linear(82944,1024),
#             nn.ReLU(),
#             nn.Linear(1024, 512),
#             nn.ReLU(),
#             nn.Linear(512,outDim)
#         )
    
    def forward(self, xb):
        return self.network(xb)

In [3]:
bigRoom = pickle.load(open("../../Analysis/Util/bigEnufSecondTable.p", "rb"))

# Available Mics: [0,1,3,4,5]
CP_LIST = bigRoom["cp_list"]
ROOM_LIST = bigRoom["room_list"]

# Available Mics: [0,1,2,3,4,5]
square_DOA = bigRoom["square_DOA"]
square_Matrix = bigRoom["square_Matrix"]

square_raw = bigRoom["sqRaw"]
cp_raw = bigRoom["cpRaw"]

# Available Mics: [0,1,2,3,4,5]
rectangle_DOA = bigRoom["rectangle_DOA"]
rectangle_Matrix = bigRoom["rectangle_Matrix"]

# Available Mics: [0,1,2,3,4,5]
table1_DOA = bigRoom["table1_DOA"]
table1_Matrix = bigRoom["table1_Matrix"]

# Available Mics: [0,1,2,3,4,5]
table2_DOA = bigRoom["table2_DOA"]
table2_Matrix = bigRoom["table2_Matrix"]

# Available Mics: [0,1,2,3,4,5]
table3_DOA = bigRoom["table3_DOA"]
table3_Matrix = bigRoom["table3_Matrix"]

# Available Mics: [0,1,2,3,4,5]
table4_DOA = bigRoom["table4_DOA"]
table4_Matrix = bigRoom["table4_Matrix"]

CP_INDICES = [i for i in range(24)]

np.vstack([ROOM_LIST[i] for i in CP_INDICES])

In [5]:
print("CP _ 1 = ", CP_LIST[0].shape)

CP _ 1 =  (400, 15)


In [6]:
print("ROOM_LIST: ", ROOM_LIST[0].shape)

ROOM_LIST:  (1, 2)


In [13]:

for idx, cPoints in enumerate(CP_LIST) :
    numSamples = cPoints.shape[0]
    print(idx, "Number of samples in CPoints: ", numSamples)
    room_sampled = np.vstack([ROOM_LIST[idx] for i in range(numSamples)])
    
    if int(idx) == 0:
        cp_stacked = cPoints
        room_stacked = room_sampled
    else :
        cp_stacked = np.vstack([cp_stacked, cPoints])
        room_stacked = np.vstack([room_stacked, room_sampled])
    
    
print("cp_stacked shape: ", cp_stacked.shape)
print("room_stacked shape: ", room_stacked.shape)
print("room list length = ", len(ROOM_LIST))

0 Number of samples in CPoints:  400
1 Number of samples in CPoints:  400
2 Number of samples in CPoints:  400
3 Number of samples in CPoints:  400
4 Number of samples in CPoints:  400
5 Number of samples in CPoints:  400
6 Number of samples in CPoints:  400
7 Number of samples in CPoints:  400
8 Number of samples in CPoints:  400
9 Number of samples in CPoints:  400
10 Number of samples in CPoints:  400
11 Number of samples in CPoints:  400
12 Number of samples in CPoints:  400
13 Number of samples in CPoints:  400
14 Number of samples in CPoints:  400
15 Number of samples in CPoints:  400
16 Number of samples in CPoints:  400
17 Number of samples in CPoints:  400
18 Number of samples in CPoints:  400
19 Number of samples in CPoints:  400
20 Number of samples in CPoints:  400
21 Number of samples in CPoints:  400
22 Number of samples in CPoints:  400
23 Number of samples in CPoints:  400
cp_stacked shape:  (9600, 15)
room_stacked shape:  (9600, 2)
room list length =  24


In [52]:
import torch.utils.data

class calibrationPoints(torch.utils.data.Dataset) :
    def __init__(self, cp_list, room_list, transforms) :
        for idx, cPoints in enumerate(cp_list) :
            numSamples = cPoints.shape[0]
#             print(idx, "Number of samples in CPoints: ", numSamples)
            room_sampled = np.vstack([ROOM_LIST[idx] for i in range(numSamples)])

            if int(idx) == 0:
                self.cp_stacked = cPoints
                self.room_stacked = room_sampled
            else :
                self.cp_stacked = np.vstack([self.cp_stacked, cPoints])
                self.room_stacked = np.vstack([self.room_stacked, room_sampled])
                
        self.transforms = transforms
        
    def __getitem__(self, idx):
        cp = self.cp_stacked[idx, :]
        roomPoint = self.room_stacked[idx, :]
        
        return cp, roomPoint
        
    def __len__(self) :
        return len(self.cp_stacked)

In [18]:
torch.utils

<module 'torch.utils' from '/usr/local/lib/python3.6/dist-packages/torch/utils/__init__.py'>

In [53]:
cPoints = calibrationPoints(CP_LIST, ROOM_LIST, None)

In [21]:
print("cPoints", cPoints[0])


cPoints (array([-0.1225  , -0.653625,  0.746875,  0.13575 ,  0.103125,  0.985   ,
        0.685   ,  0.136   ,  0.7155  ,  0.702875, -0.145875,  0.69625 ,
       -0.463   , -0.411625,  0.784875]), array([70., 70.]))


In [35]:
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable

__all__ = ['get_mean_and_std', 'init_params', 'mkdir_p', 'AverageMeter']


def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = trainloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)

    mean = torch.zeros(15, dtype=torch.float64)
    std = torch.zeros(15, dtype=torch.float64)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(15):
#             print("inputs shape: ", inputs.shape)
            mean[i] += inputs[:,i].mean()
            std[i] += inputs[:,i].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

In [36]:
mean, std = get_mean_and_std(cPoints)
print("Mean : ", mean)
print("STD  : ", std)

==> Computing mean and std..
Mean :  tensor([-0.4882, -0.1784,  0.6722,  0.4819, -0.1172,  0.8130,  0.2071,  0.4055,
         0.8004,  0.2610, -0.0434,  0.8733, -0.0186, -0.5051,  0.7604],
       dtype=torch.float64)
STD  :  tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       dtype=torch.float64)


In [46]:
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader

net = simpleMapper()
batch_size=128

torch.manual_seed(1)

val_size = 1000
train_size = len(cPoints) - val_size
train_ds, val_ds = random_split(cPoints, [train_size, val_size])
len(train_ds), len(val_ds)

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
# test_loader = DataLoader(test_dataset, batch_size*2, num_workers=4, pin_memory=True)

In [48]:
imgs, lbls = next(iter(train_loader))
imgs[7].data.shape

torch.Size([15])

In [54]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 15, 100, 2

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')


learning_rate = 1e-4
for t in range(5000):
    total_loss = 0
    for DOAs, coord in train_loader :
    
        DOAs, coord = DOAs.type(torch.FloatTensor), coord.type(torch.FloatTensor)
        # Forward pass: compute predicted y by passing x to the model. Module objects
        # override the __call__ operator so you can call them like functions. When
        # doing so you pass a Tensor of input data to the Module and it produces
        # a Tensor of output data.
        y_pred = model(DOAs)

        # Compute and print loss. We pass Tensors containing the predicted and true
        # values of y, and the loss function returns a Tensor containing the
        # loss.
        loss = loss_fn(y_pred, coord)
        total_loss += loss.item()

        # Zero the gradients before running the backward pass.
        model.zero_grad()

        # Backward pass: compute gradient of the loss with respect to all the learnable
        # parameters of the model. Internally, the parameters of each Module are stored
        # in Tensors with requires_grad=True, so this call will compute gradients for
        # all learnable parameters in the model.
        loss.backward()

        # Update the weights using gradient descent. Each parameter is a Tensor, so
        # we can access its gradients like we did before.
        with torch.no_grad():
            for param in model.parameters():
                param -= learning_rate * param.grad
                
    if t % 100 == 99:
        print(t, total_loss)

99 61643795.96875
199 61644143.84375
299 61645044.328125
399 61644972.015625
499 61643758.5
599 61645255.625
699 61646209.4375
799 61646852.390625
899 61642479.484375
999 61643627.015625
1099 61640102.578125
1199 61642705.75
1299 61645965.96875
1399 61645839.015625
1499 61645895.140625
1599 61645010.078125
1699 61645244.609375
1799 61638671.515625


KeyboardInterrupt: 